In [ ]:
# import pandas, numpy

import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', 200)

In [ ]:
# read in data

allstar = pd.read_csv("./baseballdatabank-2019.2/core/AllstarFull.csv")
appear = pd.read_csv("./baseballdatabank-2019.2/core/Appearances.csv")
awardplay = pd.read_csv("./baseballdatabank-2019.2/core/AwardsPlayers.csv")
batting = pd.read_csv("./baseballdatabank-2019.2/core/Batting.csv")
battingpost = pd.read_csv("./baseballdatabank-2019.2/core/BattingPost.csv")
fielding = pd.read_csv("./baseballdatabank-2019.2/core/Fielding.csv")

#fieldingof = pd.read_csv("./baseballdatabank-2019.2/core/FieldingOF.csv")

fieldingofsplit = pd.read_csv("./baseballdatabank-2019.2/core/FieldingOFsplit.csv")
fieldingpost = pd.read_csv("./baseballdatabank-2019.2/core/FieldingPost.csv")


hof = pd.read_csv("./baseballdatabank-2019.2/core/HallOfFame.csv")
hof.drop(columns=['votedBy', 'ballots', 'needed', 'votes', 'needed_note'], axis=1, inplace=True)

people = pd.read_csv("./baseballdatabank-2019.2/core/People.csv")
people.drop(columns = ['birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathCountry', 'deathDay', 'deathState', 'deathCity', 'nameGiven'], axis=1, inplace=True)

# pitch = pd.read_csv("./baseballdatabank-2019.2/core/Pitching.csv")
# pitchpost = pd.read_csv("./baseballdatabank-2019.2/core/PitchingPost.csv")
# salaries = pd.read_csv("./baseballdatabank-2019.2/core/Salaries.csv")
fangraph = pd.read_csv("./FanGraphs Leaderboard.csv")

In [ ]:
# drop people who have not ended their careers or who never played a game

debutnotnull = people[people['debut'].notnull()]
finalgamenotnull = people[people['finalGame'].notnull()]
people = pd.concat([debutnotnull,finalgamenotnull]).drop_duplicates()

# make people debut and final game columns into dates
people['debut'] = pd.to_datetime(people['debut'])
people['finalGame'] = pd.to_datetime(people['finalGame'])

# get difference between debuts and final games
people['careerlength'] = people['finalGame'] - people['debut']

# converts careerlength to # days
#people['careerlength'] = people['careerlength'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
people['careerlength'] = pd.to_timedelta(people['careerlength']).map(lambda x: np.nan if pd.isnull(x) else x.days)

people['recentretire'] = dt.datetime.now() - people['finalGame']
#people['recentretire'] = people['recentretire'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
people['recentretire'] = pd.to_timedelta(people['recentretire']).map(lambda x: np.nan if pd.isnull(x) else x.days)

# get careers > about 7 years
people = people[people['careerlength'] > 2000]

# remove those who retired < 5 years ago
people = people[people['recentretire'] > 1825]

people['Name'] = people['nameFirst'] + ' ' + people['nameLast']

In [ ]:
# in our learning, we are looking at all players who didn't make the Hall, not
# just those who were voted on
# so, we can remove those from the hall table that got nos, just leaving the yes
hof = hof[hof['inducted'] == 'Y']

# remove non-players from the hall
hof = hof[hof['category'] == 'Player']

# remove players from the hall that are not in our people table
peopinhall = pd.merge(people, hof, on='playerID').drop_duplicates()['playerID']
hof = hof[hof['playerID'].isin(peopinhall)]

In [ ]:
stats = {'G' : 0, 'AB' : 0, 'R' : 0, 'H' : 0, '2B' : 0, '3B' : 0, 'HR' : 0, 'RBI' : 0, 'SB' : 0, 'CS' : 0, 'BB' : 0, 'SO' : 0, 'IBB' : 0, 'HBP' : 0, 'SH' : 0, 'SF' : 0, 'GIDP' : 0}

# replace NaNs in batting with 0s
batting.fillna(value=stats)
battingpost.fillna(value=stats)

# only get batting of people in our table
batting = batting[batting['playerID'].isin(people['playerID'])]
battingpost = battingpost[battingpost['playerID'].isin(people['playerID'])]

In [ ]:
careerstats = batting.groupby('playerID').agg({'G' : np.sum, 'AB' : np.sum, 'R' : np.sum, 'H' : np.sum, '2B': np.sum, '3B' : np.sum, 'HR' : np.sum, 'RBI' : np.sum, 'SB' : np.sum, 'CS' : np.sum, 'BB' : np.sum, 'SO' : np.sum, 'IBB' : np.sum, 'HBP' : np.sum, 'SH' : np.sum, 'SF' : np.sum, 'GIDP' : np.sum})
careerpoststats = battingpost.groupby('playerID').agg({'G' : np.sum, 'AB' : np.sum, 'R' : np.sum, 'H' : np.sum, '2B': np.sum, '3B' : np.sum, 'HR' : np.sum, 'RBI' : np.sum, 'SB' : np.sum, 'CS' : np.sum, 'BB' : np.sum, 'SO' : np.sum, 'IBB' : np.sum, 'HBP' : np.sum, 'SH' : np.sum, 'SF' : np.sum, 'GIDP' : np.sum})

# filter out at-bats or games
# filter out at-bats / game
# or filter out games
# 500 games seems like a good standard
careerstats = careerstats[careerstats['G'] >= 500]
careerstats['ABPG'] = careerstats['AB'] / careerstats['G']
careerstats = careerstats[careerstats['ABPG'] >= 1] 
carrerstats = careerstats[careerstats['AB'] >= 1000]

careerpoststats['ABPG'] = careerpoststats['AB'] / careerpoststats['G']
careerpoststats = careerpoststats[careerpoststats['ABPG'] >= 1]

# careerstats.count()
# this leaves us with 2664 players, 212 of which are in the Hall of Fame

In [ ]:
# get appearances of only people in our table
appear = appear[appear['playerID'].isin(people['playerID'])]

stats = {'G_all' : 0, 'GS' : 0, 'G_batting' : 0, 'G_defense' : 0, 'G_p' : 0, 'G_c' : 0, 'G_1b' : 0, 'G_2b' : 0, 'G_3b' : 0, 'G_ss' : 0, 'G_lf' : 0, 'G_cf' : 0, 'G_rf' : 0, 'G_of' : 0, 'G_dh' : 0, 'G_ph' : 0, 'G_pr' : 0}
appear.fillna(value=stats)

careerapp = appear.groupby('playerID').agg({'G_all' : np.sum, 'G_batting' : np.sum, 'G_defense' : np.sum, 'G_p' : np.sum, 'G_c' : np.sum, 'G_1b' : np.sum, 'G_2b' : np.sum, 'G_3b' : np.sum, 'G_ss' : np.sum, 'G_lf' : np.sum, 'G_cf' : np.sum, 'G_rf' : np.sum, 'G_of' : np.sum, 'G_dh' : np.sum, 'G_ph' : np.sum, 'G_pr' : np.sum})

careerapp = careerapp[careerapp['G_all'] >= 500]
careerapp['pitchpct'] = careerapp['G_p'] / careerapp['G_all']
careerapp = careerapp[careerapp['pitchpct'] < .75]

In [ ]:
fielding = fielding[fielding['playerID'].isin(people['playerID'])]
stats = {'G' : 0, 'GS' : 0, 'InnOuts' : 0, 'PO' : 0, 'A' : 0, 'E' : 0, 'DP' : 0, 'PB' : 0, 'WP' : 0, 'SB' : 0, 'CS' : 0, 'ZR' : 0}
fielding.fillna(value=stats)
stats = {'G' : 0, 'GS' : 0, 'InnOuts' : 0, 'PO' : 0, 'A' : 0, 'E' : 0, 'DP' : 0, 'TP' : 0, 'PB' : 0, 'SB' : 0, 'CS' : 0}
fieldingpost.fillna(value=stats)

careerfield = fielding.groupby('playerID').agg({'G' : np.sum, 'GS' : np.sum, 'InnOuts' : np.sum, 'PO' : np.sum, 'A' : np.sum, 'E' : np.sum, 'DP' : np.sum, 'PB' : np.sum, 'WP' : np.sum, 'SB' : np.sum, 'CS' : np.sum, 'ZR' : np.sum})
careerpostfield = fieldingpost.groupby('playerID').agg({'G' : np.sum, 'GS' : np.sum, 'InnOuts' : np.sum, 'PO' : np.sum, 'A' : np.sum, 'E' : np.sum, 'DP' : np.sum, 'PB' : np.sum, 'TP' : np.sum, 'SB' : np.sum, 'CS' : np.sum})

careerfield = careerfield[careerfield['G'] >= 500]

In [ ]:
# awards
awardplay = awardplay[awardplay['playerID'].isin(people['playerID'])]
awardplay['pitchtriplecrown'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Pitching Triple Crown' else 0)
awardplay['triplecrown'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Triple Crown' else 0)
awardplay['baseballmagallstar'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Baseball Magazine All-Star' else 0)
awardplay['mvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Most Valuable Player' else 0)
awardplay['tsnallstar'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN All-Star' else 0)
awardplay['tsnguidemvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Guide MVP' else 0)
awardplay['tsnmlpoty'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Major League Player of the Year' else 0)
awardplay['tsnpitcheroty'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Pitcher of the Year' else 0)
awardplay['tsnpoty'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Player of the Year' else 0)
awardplay['roty'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Rookie of the Year' else 0)
awardplay['baberuth'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Babe Ruth Award' else 0)
awardplay['lou'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Lou Gehrig Memorial Award' else 0)
awardplay['wsmvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'World Series MVP' else 0)
awardplay['cyyoung'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Cy Young Award' else 0)
awardplay['goldglove'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Gold Glove' else 0)
awardplay['fireman'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Fireman of the Year' else 0)
awardplay['asgmvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'All-Star Game MVP' else 0)
awardplay['hutch'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Hutch Award' else 0)
awardplay['clemente'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Roberto Clemente Award' else 0)
awardplay['relief'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Rolaids Relief Man Award' else 0)
awardplay['nlcsmvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'NLCS MVP' else 0)
awardplay['alcsmvp'] = awardplay['awardID'].apply(lambda x : 1 if x == 'ALCS MVP' else 0)
awardplay['silver'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Silver Slugger' else 0)
awardplay['branchrickey'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Branch Rickey Award' else 0)
awardplay['hankaaron'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Hank Arron Award' else 0)
awardplay['tsnrelief'] = awardplay['awardID'].apply(lambda x : 1 if x == 'TSN Reliever of the Year Award' else 0)
awardplay['comeback'] = awardplay['awardID'].apply(lambda x : 1 if x == 'Comeback Player of the Year' else 0)

careerawards = awardplay.groupby('playerID').agg({'pitchtriplecrown' : np.sum, 'triplecrown' : np.sum, 'baseballmagallstar' : np.sum, 
                                                 'mvp' : np.sum, 'tsnallstar' : np.sum, 'tsnguidemvp' : np.sum, 'tsnmlpoty' : np.sum,
                                                 'tsnpitcheroty' : np.sum, 'tsnpoty' : np.sum, 'roty' : np.sum, 'baberuth' : np.sum,
                                                 'lou' : np.sum, 'wsmvp' : np.sum, 'cyyoung' : np.sum, 'goldglove' : np.sum,
                                                 'fireman' : np.sum, 'asgmvp' : np.sum, 'hutch' : np.sum, 'clemente' : np.sum,
                                                 'relief' : np.sum, 'nlcsmvp' : np.sum, 'alcsmvp' : np.sum, 'silver' : np.sum,
                                                 'branchrickey' : np.sum, 'hankaaron' : np.sum, 'tsnrelief' : np.sum,
                                                 'comeback' : np.sum})

In [ ]:
fanpeop = pd.merge(people, fangraph, on='Name')
fanpeop = pd.merge(careerstats, fanpeop, on='playerID')

In [ ]:
hofers = pd.merge(people, hof, on='playerID')
hofbats = pd.merge(careerstats, hof, on='playerID')
hofpost = pd.merge(careerpoststats, hof, on='playerID')
hofapp = pd.merge(careerapp, hof, on='playerID')
hoffield = pd.merge(careerfield, hof, on='playerID')
hoffieldpost = pd.merge(careerpostfield, hof, on='playerID')
hofawards = pd.merge(careerawards, hof, on='playerID')
# hofers.count()
# 229
# hofbats.count()
# 203
# hofpost.count()
# 189
# hofapp.count()
# 160
# hoffield.count()
# 209
# hoffieldpost.count()
# 182
# hofawards.count()
# 195

In [ ]:
fanpeop.rename(index=str, columns={"G_x" : "Career Stats G", "AB" : "Career Stats AB", "R_x" : "Career Stats R", "H" : "Career Stats H",
                                  "2B" : "Career Stats 2B", "3B" : "Career Stats 3B", "HR_x" : "Career Stats HR",
                                  "RBI_x" : "Career Stats RBI", "SB_x" : "Career Stats SB", "CS" : "Career Stats CS",
                                  "BB" : "Career Stats BB", "SO" : "Career Stats SO", "IBB" : "Career Stats IBB", 
                                  "HBP" : "Career Stats HBP", "SH" : "Career Stats SH", "SF": "Career Stats SF",
                                  "GIDP" : "Career Stats GIDP", "G_y" : "FanGraphs G", "PA" : "FanGraphs PA",
                                  "HR_y" : "FanGraphs HR", "R_y" : "FanGraphs R", "RBI_y" : "FanGraphs RBI",
                                  "SB_y" : "FanGraphs SB"}, inplace=True)
fanpeop.drop(columns=['nameFirst', 'nameLast', 'debut', 'finalGame', 'retroID', 'bbrefID', 'recentretire',
                     'Name', 'Team', 'playerid'], axis=1, inplace=True)

In [ ]:
data = fanpeop
data = pd.merge(data, careerawards, on='playerID', how='left').drop_duplicates()
data.drop(columns=['ABPG', 'pitchtriplecrown', 'tsnpitcheroty', 'cyyoung', 'relief', 'tsnrelief'],axis=1, inplace=True)
data.rename(index=str, columns={'pitchtriplecrown' : "Pitching Triple Crown", 'triplecrown' : "Triple Crown",
                               'baseballmagallstar' : "Baseball Mag All-Star", 'mvp' : "MVP", 'tsnallstar' : "TSN All-Star",
                               'tsnguidemvp' : "TSN Guide MVP", 'tsnmlpoty' : "TSN ML POTY", 'tsnpoty' : "TSN POTY",
                               'roty' : "Rookie of the Year", 'baberuth' : "Babe Ruth Award", 'lou' : "Lou Gehrig Memorial Award",
                               'wsmvp' : "World Series MVP", 'goldglove' : "Gold Glove", 'fireman' : "TSN Fireman of the Year",
                               'asgmvp' : "ASG MVP", 'hutch' : "Hutch Award", 'clemente' : "Roberto Clemente Award",
                               'nlcsmvp' : "NLCS MVP", 'alcsmvp' : "ALCS MVP", 'silver' : "Silver Slugger", 
                               'branchrickey' : "Branch Rickey Award", 'hankaaron' : "Hank Aaron Award", 
                               'comeback' : "Comeback Player of the Year"}, inplace=True)

In [ ]:
data = pd.merge(data, careerfield, on='playerID', how='left').drop_duplicates()
data.drop(columns=['WP', 'SB', 'CS', 'ZR', 'PB'],axis=1, inplace=True)
data.rename(index=str, columns={'G' : 'Games Fielding', 'GS' : 'GS Fielding', 'InnOuts' : 'Inning Outs Fielding',
                               'PO' : 'PO Fielding', 'A' : 'A Fielding', 'E' : 'E Fielding',
                               'DP' : 'DP Fielding'},inplace=True)

In [ ]:
data = pd.merge(data, careerpostfield, on='playerID', how='left').drop_duplicates()
data.drop(columns=['PB', 'TP', 'CS', 'SB'], axis=1, inplace=True)
data.rename(index=str, columns={'G' : 'G Fielding Post', 'GS' : 'GS Fielding Post', 'InnOuts' : 'Inning Outs Fielding Post', 
                               'PO' : 'PO Fielding Post', 'A' : 'A Fielding Post', 'E' : 'E Fielding Post',
                               'DP' : 'DP Fielding Post'},inplace=True)

In [ ]:
data = pd.merge(data, careerpoststats, on='playerID', how='left').drop_duplicates()
data.drop(columns=['ABPG'], axis=1,inplace=True)
data.rename(index=str, columns={'G' : 'G Post', 'AB' : 'AB Post', 'R' : 'R Post', 'H' : 'H Post', '2B' : '2B Post',
                               '3B' : '3B Post', 'HR' : 'HR Post', 'RBI' : 'RBI Post' ,'SB' : 'SB Post', 'CS' : 'CS Post',
                               'BB' : 'BB Post', 'SO' : 'SO Post', 'IBB' : 'IBB Post', 'HBP' : 'HBP Post', 
                               'SH' : 'SH Post', 'SF' : 'SF Post', 'GIDP' : 'GIDP Post'},inplace=True)

In [ ]:
data = data.drop_duplicates()
data = data.fillna(0)
data['Bats'] = data['bats'].apply(lambda x : 1 if x == 'R' else 0)
data['Throws'] = data['throws'].apply(lambda x : 1 if x == 'R' else 0)
data.drop(columns=['bats', 'throws'], axis=1, inplace=True)

In [ ]:
data['K%'] = data['K%'].astype(str)
data['K%'] = data['K%'].str.replace(' %$', '')
data['BB%'] = data['BB%'].astype(str)
data['BB%'] = data['BB%'].str.replace(' %$', '')
data = data.replace('nan', 50)
data[['K%', 'BB%']] = data[['K%', 'BB%']].apply(pd.to_numeric)

In [ ]:
collist = data.columns.tolist()
data[collist[1:]] = data[collist[1:]].apply(pd.to_numeric)

In [ ]:
hoferslist = hofers.columns.tolist()
hofers = hofers[hoferslist[0]]
hoflist = hofers.tolist()
data['HOF'] = data['playerID'].apply(lambda x : 1 if x in hoflist else 0) 

In [ ]:
data

#TODO: make sure that we have the correct thresholds for stats we want